In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
import pickle 
from tensorflow.keras.models import load_model
from numpy import loadtxt
from sklearn.calibration import calibration_curve
pd.set_option('display.max_columns', None)

In [2]:
big_df = pd.read_csv("../data/big_df.csv")
big_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (215,216) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893,0.777778,0.717557,1.00,0.285714,0.474104,0.516234,0.77

In [3]:
# Dropping all fights that occured before unified rules were implamented 
df = big_df
df["date"] = pd.to_datetime(df["date"])
df = df[df["date"] >= "November 17, 2000"]
df

,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893,0.777778,0.717557,1.00,0.285714,0.474104,0.516234,0.77

In [4]:
df["red_age"] = pd.Series(df["red_age"], dtype="int")
df["blue_age"] = pd.Series(df["blue_age"], dtype="int")
df["red_heightIN"] = pd.Series(df["red_heightIN"], dtype="int")
df["blue_heightIN"] = pd.Series(df["blue_heightIN"], dtype="int")
df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893,0.777778,0.717557,1.00,0.285714,0.474104,0.516234,0.77

In [5]:
df = df[df["red_age"] < 75]
df = df[df["blue_age"] < 75]
df

,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893,0.777778,0.717557,1.000000,0.285714,0.474104,0.516234,

In [6]:
df = df[df["red_heightIN"] < 100]
df = df[df["blue_heightIN"] < 100]
df

,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893,0.777778,0.717557,1.000000,0.285714,0.474104,0.516234,

In [7]:
# removing draws and no contests 
df = df[df["winner"] != "Draw"] # it's arguable draws are useful datapoints, but I choose to remove them
df = df[df["winner"] != "NC"] # no contests however are not useful datapoints
df["winner"].value_counts()

red     3844
blue    2032
tbd       10
Name: winner, dtype: int64

In [8]:
df = df[df["method"] != "DQ"]
df["method"].value_counts()

UD            2147
KO/TKO        1951
Submission    1143
SD             574
MD              43
tbd             10
Name: method, dtype: int64

In [9]:
# isolating our future fights to predict
future_df = df[df["winner"] == "tbd"]
future_df

,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893,0.777778,0.717557,1.000000,0.285714,0.474104,0.516234,

In [10]:
# Label encode our stances and weight class 
number = LabelEncoder()
# data['red_name'] = number.fit_transform(data['red_name'].astype('str'))
# data['blue_name'] = number.fit_transform(data['blue_name'].astype('str'))
future_df['red_stance'] = number.fit_transform(future_df['red_stance'].astype('str'))
future_df['blue_stance'] = number.fit_transform(future_df['blue_stance'].astype('str'))
future_df['weight_class'] = number.fit_transform(future_df['weight_class'].astype('str'))
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893,0.777778,0.717557,1.000000,0.285714,0.474104,0.516234,

In [11]:
# drop columns
data = future_df.drop(columns=["lev1", "winner", "red_name", "blue_name", "date", "event", "red_DOB", "blue_DOB", "method",\
                               "location", "red_nickname", "blue_nickname", "red_weight", "blue_weight", "index"])
winners = df["winner"]
feature_names = data.columns

In [12]:
X_scaler = MinMaxScaler().fit(data)
X_scaled = X_scaler.transform(data)

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(winners)
encoded_y = label_encoder.transform(winners)

# Step 2: Convert encoded labels to one-hot-encoding
y_categorical = to_categorical(encoded_y)

In [13]:
# load in our saved random forest model...
loaded_model = load_model("../saved_models/neural_up_model.h5")


In [14]:
# pred = loaded_model.predict(X_scaled)
# future_df["prediction"] = pred
# future_df

In [15]:

# pred = loaded_model.predict(data)
encoded_predictions = loaded_model.predict_classes(X_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
prediction_labels


array(['blue', 'blue', 'blue', 'blue', 'red', 'blue', 'red', 'blue',
       'red', 'red'], dtype=object)

In [16]:
future_df["prediction"] = prediction_labels
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index,prediction
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893,0.777778,0.717557,1.000000,0.285714,0.47410

In [24]:
pcts = loaded_model.predict_proba(X_scaled)
favs = []
dogs = []
for pct in pcts:
    print(pct)
    fav = max(pct)
    dog = min(pct)
    favs.append(fav)
    dogs.append(dog)

[9.9999881e-01 1.1541118e-06]
[1.0000000e+00 1.2010409e-09]
[1.0000000e+00 8.0985496e-10]
[1.0000000e+00 2.2566444e-19]
[0. 1.]
[9.999188e-01 8.117428e-05]
[0.0187085  0.98129153]
[1.00000000e+00 1.10407924e-20]
[5.3118629e-04 9.9946886e-01]
[9.898649e-19 1.000000e+00]


In [18]:
future_df["fav_pct"] = favs
future_df["dog_pct"] = dogs
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index,prediction,fav_pct,dog_pct
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893,0.777778,0.717557,1.000000,

In [19]:
future_df["fav_pct"] = future_df["fav_pct"].round(2)
future_df["dog_pct"] = future_df["dog_pct"].round(2)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index,prediction,fav_pct,dog_pct
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893,0.777778,0.717557,1.000000,

In [20]:
future_df["fav_moneyline"] = (future_df["fav_pct"] * 100) / (100 - (future_df["fav_pct"] * 100)) * (-100)
future_df["dog_moneyline"] = ((100 - (future_df["dog_pct"] * 100)) / (future_df["dog_pct"] * 100) * 100)
future_df

C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\audri\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,lev1,blue_CtrlDiff,red_CtrlDiff,blue_CtrlSecPerMin,red_CtrlSecPerMin,blue_CtrldSecPerMin,red_CtrldSecPerMin,blue_DOB,red_DOB,blue_KDsPer100Stk,red_KDsPer100Stk,blue_NCs,red_NCs,blue_SigBodyStkAcc,red_SigBodyStkAcc,blue_SigBodyStkDef,red_SigBodyStkDef,blue_SigClchStkAcc,red_SigClchStkAcc,blue_SigClchStkDef,red_SigClchStkDef,blue_SigDistStkAcc,red_SigDistStkAcc,blue_SigDistStkDef,red_SigDistStkDef,blue_SigGndStkAcc,red_SigGndStkAcc,blue_SigGndStkDef,red_SigGndStkDef,blue_SigHeadStkAcc,red_SigHeadStkAcc,blue_SigHeadStkDef,red_SigHeadStkDef,blue_SigLegStkAcc,red_SigLegStkAcc,blue_SigLegStkDef,red_SigLegStkDef,blue_SigStkAbsPerMin,red_SigStkAbsPerMin,blue_SigStkAcc,red_SigStkAcc,blue_SigStkDef,red_SigStkDef,blue_SigStkPerMin,red_SigStkPerMin,blue_SigStkPerMinDiff,red_SigStkPerMinDiff,blue_StkBodyAbsPerMin,red_StkBodyAbsPerMin,blue_StkBodyPerMin,red_StkBodyPerMin,blue_StkClchAbsPerMin,red_StkClchAbsPerMin,blue_StkClchPerMin,red_StkClchPerMin,blue_StkDistAbsPerMin,red_StkDistAbsPerMin,blue_StkDistPerMin,red_StkDistPerMin,blue_StkGndAbsPerMin,red_StkGndAbsPerMin,blue_StkGndPerMin,red_StkGndPerMin,blue_StkHeadAbsPerMin,red_StkHeadAbsPerMin,blue_StkHeadPerMin,red_StkHeadPerMin,blue_StkLegAbsPerMin,red_StkLegAbsPerMin,blue_StkLegPerMin,red_StkLegPerMin,blue_TDAcc,red_TDAcc,blue_TDDef,red_TDDef,blue_TDsAbsPerMin,red_TDsAbsPerMin,blue_TDsPerMin,red_TDsPerMin,blue_TotStkAbsPerMin,red_TotStkAbsPerMin,blue_TotStkAcc,red_TotStkAcc,blue_TotStkDef,red_TotStkDef,blue_TotStkPerMin,red_TotStkPerMin,blue_TotStkPerMinDiff,red_TotStkPerMinDiff,blue_age,red_age,blue_draws,red_draws,blue_fights,red_fights,blue_heightIN,red_heightIN,blue_losses,red_losses,blue_name,red_name,blue_nickname,red_nickname,blue_reachIN,red_reachIN,blue_stance,red_stance,blue_sumCtrlSec,red_sumCtrlSec,blue_sumCtrldSec,red_sumCtrldSec,blue_sumDQLoss,red_sumDQLoss,blue_sumDQWin,red_sumDQWin,blue_sumDecLoss,red_sumDecLoss,blue_sumDecWin,red_sumDecWin,blue_sumFightSec,red_sumFightSec,blue_sumKDs,red_sumKDs,blue_sumKOLoss,red_sumKOLoss,blue_sumKOWin,red_sumKOWin,blue_sumKedD,red_sumKedD,blue_sumReversed,red_sumReversed,blue_sumRevs,red_sumRevs,blue_sumSigBodyAbs,red_sumSigBodyAbs,blue_sumSigBodyAtt,red_sumSigBodyAtt,blue_sumSigBodyLnd,red_sumSigBodyLnd,blue_sumSigBodySeen,red_sumSigBodySeen,blue_sumSigClchAbs,red_sumSigClchAbs,blue_sumSigClchAtt,red_sumSigClchAtt,blue_sumSigClchLnd,red_sumSigClchLnd,blue_sumSigClchSeen,red_sumSigClchSeen,blue_sumSigDistAbs,red_sumSigDistAbs,blue_sumSigDistAtt,red_sumSigDistAtt,blue_sumSigDistLnd,red_sumSigDistLnd,blue_sumSigDistSeen,red_sumSigDistSeen,blue_sumSigGndAbs,red_sumSigGndAbs,blue_sumSigGndAtt,red_sumSigGndAtt,blue_sumSigGndLnd,red_sumSigGndLnd,blue_sumSigGndSeen,red_sumSigGndSeen,blue_sumSigHeadAbs,red_sumSigHeadAbs,blue_sumSigHeadAtt,red_sumSigHeadAtt,blue_sumSigHeadLnd,red_sumSigHeadLnd,blue_sumSigHeadSeen,red_sumSigHeadSeen,blue_sumSigLegAbs,red_sumSigLegAbs,blue_sumSigLegAtt,red_sumSigLegAtt,blue_sumSigLegLnd,red_sumSigLegLnd,blue_sumSigLegSeen,red_sumSigLegSeen,blue_sumSigStksAbs,red_sumSigStksAbs,blue_sumSigStksAtt,red_sumSigStksAtt,blue_sumSigStksLnd,red_sumSigStksLnd,blue_sumSigStksSeen,red_sumSigStksSeen,blue_sumSubAtt,red_sumSubAtt,blue_sumSubEsc,red_sumSubEsc,blue_sumSubLoss,red_sumSubLoss,blue_sumSubWin,red_sumSubWin,blue_sumTDsAbs,red_sumTDsAbs,blue_sumTDsAtt,red_sumTDsAtt,blue_sumTDsLnd,red_sumTDsLnd,blue_sumTDsSeen,red_sumTDsSeen,blue_sumTitleFights,red_sumTitleFights,blue_sumTotStksAbs,red_sumTotStksAbs,blue_sumTotStksAtt,red_sumTotStksAtt,blue_sumTotStksLnd,red_sumTotStksLnd,blue_sumTotStksSeen,red_sumTotStksSeen,blue_weight,red_weight,blue_winPCT,red_winPCT,blue_wins,red_wins,blue_SoS,red_SoS,winner,weight_class,title_fight,event,date,method,location,index,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,0,7.727499,10.163043,13.754351,26.630435,6.026852,16.467391,1990-10-01,1995-03-01,0.598802,0.520833,0,0,0.722222,0.823529,0.500000,0.105882,0.733333,0.673913,0.272727,0.212121,0.496212,0.394118,0.702602,0.575893

In [21]:
future_df["fav_moneyline"] = future_df["fav_moneyline"].astype(int)
future_df["dog_moneyline"] = future_df["dog_moneyline"].astype(int)
future_df

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
future_df.to_csv("data/neaural_future_preds.csv")

In [ ]:
my_picks = future_df[["blue_name", "red_name", "prediction", "fav_pct", "dog_pct", "fav_moneyline", "dog_moneyline"]]
my_picks.to_csv("data/neaural_picks.csv")
my_picks